# Patrones de diseño en Python
## Introducción a los patrones de diseño y qué herramientas brinda Python para implementarlos

---

### Luciano Serruya Aloisi
#### 30 de Septiembre, 2017

***

##### Según GoF, un _Patrón de Diseño_ se define como:

> _Descripciones de clases y objetos relacionados que están particularizados para resolver un problema de diseño general en un determinado contexto_

***

***
>_Each pattern describes a problem which occurs over and
over again in our environment, and then describes the core of the solution to that
problem, in such a way that you can use this solution a million times over, without ever
doing it the same way twice_

_Christopher Alexander, Sara Ishikawa, Murray Silverstein, Max Jacobson,
Ingrid Fiksdahl-King, and Shlomo Angel. A Pattern Language. Oxford
University Press, New York, 1977._

### Incorporar las clases de PD (Estructurales, Creacionales, Comportamiento)
### Hablar sobre First-class objects, Closures, y capaz que bounded y unbounded methods 
##### (aunque los unbounded methods no existen más en Python3, ahora son funciones)

### Poner un PD de cada clase
### Agregar memes obvio

### GoF organiza los patrones por _propósito_, y por _alcance_

### Categorías por _propósito_
* Creacionales: se encargan de la instanciación de objetos
* Comportamiento: caracterizan las formas en que los objetos y las clases interactúan entre sí y cómo distribuyen la responsabilidad
* Estructurales: se encargan de cómo se componen las clases y los objetos
***

***
### Categorías por _alcance_
* Clase: especifican relaciones entre clases y sus clases hijas. Relación estática a través de la **herencia**
* Objeto: especifican relaciones entre objetos, dichos objetos pueden ser cambiados en tiempo de ejecución. Resultados más dinámicos a través de la **composición** y **delegación**

- [x] Finish my changes
